In [1]:
import os
import warnings
from datetime import datetime

import matplotlib
matplotlib.use('agg')

import numpy as np
import pyart
from matplotlib import pyplot as plt
import cftime

import radar_codes
import filtering
import phase
import hydrometeors
import attenuation
import rainrate
import file_util
import gridding

import dask
import dask.bag as db
from dask.diagnostics import ProgressBar

warnings.simplefilter('ignore')



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
#config
vol_root = '/g/data/rq0/level_1/odim_pvol'
cf_root = '/g/data/kl02/jss548/PST/dprain/dprain-retreival/cfradial'
rf_root = '/g/data/kl02/jss548/PST/dprain/dprain-retreival/rfgrid'
img_root = '/g/data/kl02/jss548/PST/dprain/dprain-retreival/img'
VERBOSE = True

rid = 31
date_str = '20210119'

#rid = 71
#date_str = '20200119'
#date_str = '20200911'
# rid = 66
# date_str = '20200309'

NCPU = 10

refl_offset = 0
zdr_offset = 0

if refl_offset != 0 or zdr_offset != 0:
    print('WARNING: IN CALIBRATION TESTING MODE')

In [3]:
def buffer(vol_ffn):
    try:
        torrentfields(vol_ffn)
    except Exception as e:
        print('failed on', vol_ffn,'with',e)

In [4]:
def torrentfields(vol_ffn):

    print('processing', vol_ffn)

    #read radar volume
    radar = pyart.aux_io.read_odim_h5(vol_ffn, file_field_names=True)
    #get time
    valid_time = cftime.num2pydate(radar.time['data'][0], radar.time['units'])

    #get radar band
    wavelength = radar_codes.get_wavelength(vol_ffn)
    if wavelength<8:
        band = 'C'
    else:
        band = 'S'
    if VERBOSE:
        print('band', band)
        
    ##################################################################################################
    #
    # Preprocessing
    #
    ##################################################################################################

    #offset refl if required for testing
    refl = radar.fields['DBZH']['data'] + refl_offset
    radar.add_field_like('DBZH', 'DBZH', refl, replace_existing=True)
    zdr = radar.fields['ZDR']['data'] + zdr_offset
    radar.add_field_like('ZDR', 'ZDR', zdr, replace_existing=True)
    
    # Correct RHOHV
    rho_corr = radar_codes.correct_rhohv(radar, snr_name='SNRH')
    radar.add_field_like('RHOHV', 'RHOHV_CORR', rho_corr, replace_existing=True)

    # Correct ZDR
    corr_zdr = radar_codes.correct_zdr(radar, snr_name='SNRH')
    radar.add_field_like('ZDR', 'ZDR_CORR', corr_zdr, replace_existing=True)

#     from importlib import reload
#     reload(radar_codes)
    
    # Temperature    
    height, temperature, isom = radar_codes.temperature_profile_access(radar, source='access')
    radar.add_field('temperature', temperature, replace_existing=True)
    radar.add_field('height', height, replace_existing=True)
    radar.add_field('height_over_isom', isom, replace_existing=True)

    # GateFilter
    gatefilter = filtering.do_gatefilter(radar,
                                         refl_name='DBZH',
                                         phidp_name="PHIDP",
                                         rhohv_name='RHOHV_CORR',
                                         zdr_name="ZDR_CORR",
                                         snr_name='SNRH')
    #create fake NCP
    ncp = pyart.config.get_metadata('normalized_coherent_power')
    ncp['data'] = np.zeros_like(radar.fields['RHOHV']['data'])
    ncp['data'][gatefilter.gate_included] = 1
    radar.add_field('NCP', ncp, replace_existing=True)

    # phidp filtering
    phidp, kdp = phase.phidp_giangrande(radar, gatefilter, rhv_field='RHOHV_CORR', refl_field='DBZH')
    radar.add_field('PHIDP_VAL', phidp, replace_existing=True)
    radar.add_field('KDP_VAL', kdp, replace_existing=True)
    kdp_field_name = 'KDP_VAL'
    phidp_field_name = 'PHIDP_VAL'

    #first try to use exisiting NCAR HCA into a field. Sometimes it is missing due a missing DP fields.
    try:
        hca_field = hydrometeors.extract_ncar_pid(radar, vol_ffn)
    except:
        #insert CSU HCA if NCAR PID is not in the file
        hca_field = hydrometeors.csu_hca(radar,
                                          gatefilter,
                                          kdp_name=kdp_field_name,
                                          zdr_name='ZDR_CORR',
                                          rhohv_name='RHOHV_CORR',
                                          refl_name='DBZH',
                                          band=band)
    radar.add_field('radar_echo_classification', hca_field, replace_existing=True)
    
    ##################################################################################################
    #
    # Retrievals
    #
    ##################################################################################################
#     from importlib import reload
#     reload(rainrate)
#     reload(attenuation)

    #index index of lowest sweep
    first_sweep_idx = np.argmin(radar.fixed_angle['data'])
    #estimate alpha
    alpha, alpha_method = attenuation.estimate_alpha_zhang2020(radar, band, first_sweep_idx,
                                           refl_field='DBZH', zdr_field='ZDR_CORR', rhohv_field='RHOHV_CORR',
                                           verbose=VERBOSE)

    #estimate specific attenuation
    radar = attenuation.retrieve_zphi(radar, band, alpha=alpha, alpha_method=alpha_method,
                                     refl_field='DBZH', phidp_field=phidp_field_name, rhohv_field='RHOHV_CORR')

    #estimate rainfall
    radar = rainrate.conventional(radar, alpha=92, beta=1.7, refl_field='corrected_reflectivity')
    radar = rainrate.polarimetric(radar, band, refl_field='corrected_reflectivity',
                                  kdp_field=kdp_field_name, phidp_field=phidp_field_name, rhohv_field='RHOHV_CORR', min_delta_phidp=1.)
    
    ##################################################################################################
    #
    # Write outputs CF Radial
    #
    ##################################################################################################
    
    #create paths
    cf_path = f'{cf_root}/{rid:02}/{date_str}'
    if not os.path.exists(cf_path):
        os.makedirs(cf_path)
    cf_fn = f'{rid:02}_{valid_time.strftime("%Y%m%d_%H%M%S")}.vol.nc' #this filename should match
    cf_ffn = f'{cf_path}/{cf_fn}'
    #write to cf
    pyart.io.write_cfradial(cf_ffn, radar)
    
    ##################################################################################################
    #
    # Create and write grid
    #
    ##################################################################################################
        
    # grid first two sweeps (second sweep used as a fallback where the lower grid has no data)
    sort_idx = np.argsort(radar.fixed_angle['data'])
    data_sweep1 = radar.get_field(sort_idx[0], 'hybrid_rainrate', copy=True).filled(np.nan)
    data_sweep2 = radar.get_field(sort_idx[1], 'hybrid_rainrate', copy=True).filled(np.nan)
    data_combined = np.nanmax(np.stack((data_sweep1, data_sweep2), axis=2), axis=2)
    #build metadata and grid
    r = radar.range['data']
    th = 450 - radar.get_azimuth(sort_idx[0], copy=False)
    th[th < 0] += 360
    R, A = np.meshgrid(r, th)
    x = R * np.cos(np.pi * A / 180)
    y = R * np.sin(np.pi * A / 180)
    xgrid = np.linspace(-127750,127750,512)
    xgrid, ygrid = np.meshgrid(xgrid, xgrid)
    gatespacing = r[1]-r[0]
    rain_grid_2d = gridding.KDtree_nn_interp(data_combined, x, y, xgrid, ygrid, nnearest = 16, maxdist = 2500)
    
    #extract metadata for RF3 grids
    standard_lat_1, standard_lat_2 = file_util.rf_standard_parallel_lookup(rid)
    if standard_lat_1 is None:
        print('failed to lookup standard parallels')
    rf_lon0, rf_lat0 = file_util.rf_grid_centre_lookup(rid)
    if rf_lon0 is None:
        print('failed to lookup rf grid centre coordinates')
    
    #create paths
    rf_path = f'{rf_root}/{rid:02}/{date_str}'
    if not os.path.exists(rf_path):
        os.makedirs(rf_path)
    rf_fn = f'{rid}_{valid_time.strftime("%Y%m%d_%H%M%S")}.prcp-rrate.nc' #this filename should match
    rf_ffn = f'{rf_path}/{rf_fn}'
    
    #write to nc
    file_util.write_rf_nc(rf_ffn, rid, valid_time.timestamp(), rain_grid_2d, rf_lon0, rf_lat0, (standard_lat_1, standard_lat_2))
    
    #create image and save to file
    ###################################################################################################################
    tilt = sort_idx[0] #first tilt
    ylim = [-128, 128]
    xlim = [-128, 128]

    fig = plt.figure(figsize=[16,8])
    display = pyart.graph.RadarDisplay(radar)

    ax = plt.subplot(231)
    display.plot_ppi('DBZH', tilt, vmin=0, vmax=60, cmap='pyart_HomeyerRainbow')
    ax.set_xlabel('')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax = plt.subplot(232)
    display.plot_ppi('KDP_VAL', tilt, vmin=0, vmax=4, cmap='pyart_HomeyerRainbow')
    ax.set_xlabel('')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax = plt.subplot(233)
    display.plot_ppi('PHIDP_VAL', tilt, vmin=0, vmax=25, cmap='pyart_HomeyerRainbow')
    ax.set_xlabel('')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax = plt.subplot(234)
    display.plot_ppi('zr_rainrate', tilt, vmin=0.2, vmax=75, cmap='pyart_RRate11', title='SP retrieval')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax = plt.subplot(235)
    display.plot_ppi('hybrid_rainrate', tilt, vmin=0.2, vmax=75, cmap='pyart_RRate11', title='DP retrieval')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    ax = plt.subplot(236)
    ax.set_title('DP gridded data')
    img = plt.imshow(np.flipud(rain_grid_2d), vmin=0, vmax=75, cmap=pyart.graph.cm._generate_cmap('RRate11',100))
    cbar = fig.colorbar(img, ax=ax)
    cbar.set_label('DP rain rate (mm/hr)')
    
    img_path = f'{img_root}/{rid:02}/{date_str}'
    if not os.path.exists(img_path):
        os.makedirs(img_path)
    img_fn = f'{rid:02}_{valid_time.strftime("%Y%m%d_%H%M%S")}.jpg' #this filename should match
    img_ffn = f'{img_path}/{img_fn}'
    plt.savefig(img_ffn, dpi=100)
    fig.clf()
    plt.close()
    ###################################################################################################################

In [5]:
def manager(date_str, rid):

    #unpack and list daily zip
    vol_zip = f'{vol_root}/{rid:02}/{date_str[0:4]}/vol/{rid:02}_{date_str}.pvol.zip'
    temp_dir = True
    vol_ffn_list = file_util.unpack_zip(vol_zip)

    
#     for vol_ffn in vol_ffn_list[60:]:
#         torrentfields(vol_ffn)
    #run retrieval
    i            = 0
    n_files      = len(vol_ffn_list)   
    for flist_chunk in file_util.chunks(vol_ffn_list, NCPU): #CUSTOM RANGE USED
        bag = db.from_sequence(flist_chunk).map(buffer)
        _ = bag.compute()
        i += NCPU
        del bag
        print('processed: ' + str(round(i/n_files*100,2)))
        
    #clean up
    temp_vol_dir = os.path.dirname(vol_ffn_list[0])
    if '/tmp' in temp_vol_dir:
        os.system('rm -rf '+ temp_vol_dir)

In [ ]:
manager(date_str, rid)

In [6]:
#unpack and list daily zip
vol_zip = f'{vol_root}/{rid:02}/{date_str[0:4]}/vol/{rid:02}_{date_str}.pvol.zip'
temp_dir = True
vol_ffn_list = file_util.unpack_zip(vol_zip)

In [ ]:
for vol_ffn in vol_ffn_list:
    torrentfields(vol_ffn)

processing /jobfs/22921372.gadi-pbs/tmps061eyq6/31_20210119_000029.pvol.h5
band C
defaults 0.07851402206955327 0.09803630895779822
temp: 17.92802456408414 C


case 1 valid bins 0
case 1 thresholds 8
case 1a valid bins 0
case 1a thresholds 3
case 2 valid bins 0
case 2 thresholds 11
case 3 valid bins 0
case 3 thresholds 9
case 4 False
case 4 total count 0.0
case 4 min 50

5: default stratiform
alpha value 0.09803630895779822
processing /jobfs/22921372.gadi-pbs/tmps061eyq6/31_20210119_000529.pvol.h5
band C
defaults 0.07510024044738413 0.09427681712173652
temp: 19.48292395928757 C


case 1 valid bins 0
case 1 thresholds 8
case 1a valid bins 0
case 1a thresholds 3
case 2 valid bins 0
case 2 thresholds 11
case 3 valid bins 0
case 3 thresholds 9
case 4 False
case 4 total count 0.0
case 4 min 50

5: default stratiform
alpha value 0.09427681712173652
processing /jobfs/22921372.gadi-pbs/tmps061eyq6/31_20210119_001029.pvol.h5
band C
defaults 0.0742361680572847 0.09332524099418063
temp: 19.8764891